In [1]:
import pandas as pd
import numpy as np
import random
import re
import xlwt
import copy

In [2]:
# read excel
df_test = pd.read_excel("possiblematch.xlsx")
df = df_test.copy()

preference_dict = {}

for i in range(len(df)):
    if df.loc[i, "Subject"] not in preference_dict:
        preference_dict[df.loc[i, "Subject"]] = {}
        preference_dict[df.loc[i, "Subject"]][df.loc[i, "Match"]] = df.loc[i, "Score"]
    else:
        preference_dict[df.loc[i, "Subject"]][df.loc[i, "Match"]] = df.loc[i, "Score"]

In [3]:
# everyone can be the subject
subject_pref = copy.deepcopy(preference_dict)

# successfully paired dict
paired = {}


In [4]:
def getCurrMatch (pep, paired):
    inv_paired = {v: k for k, v in paired.items()}
    # if the pep is not matched, return -1
    curr_match = -1
    if pep in paired:
        curr_match = paired[pep]
    elif pep in inv_paired:
        curr_match = inv_paired[pep]
        
    return curr_match

In [5]:
def overallScore (preference_dict, paired):
    scoreSum = 0
    for pep in list(paired.keys()):
        sub = preference_dict[pep][paired[pep]]
        rec = preference_dict[paired[pep]][pep]
        scoreSum += sub + rec
    return scoreSum

In [6]:
def findPair (subject_pref, paired):
    # initialize received proposal list for this iteration
    received = {}
    for i in range(len(preference_dict)):
        received[i] = []
    
    #subject side
    for pep in list(subject_pref.keys()):
        # if the subject's list to send proposal is not empty
        if subject_pref[pep]:
            # only single subject sends out proposal to the first one in the preference
            if getCurrMatch(pep, paired) == -1:
                target = list(subject_pref[pep].keys())[0]
                received[target].append(pep)
                # after sending invitation, the first target will be deleted
                del subject_pref[pep][target]
    
    # receiver side
    for rece in list(received.keys()):
        # if in this iteration this receiver got invitation
        if received[rece]:
            # single receiver will view all of the invitation and select the best one to pair
            if getCurrMatch(rece, paired) == -1:
                # find the ones who send the invitation from the possible matches
                poss_invi = []
                for i in received[rece]:
                    if i in list(preference_dict[rece].keys()):
                        poss_invi.append(i)
            
                # find the best one among the possible choices in the invitations
                if poss_invi:
                    current_best = poss_invi[0]
                    for invi in poss_invi:
                        if preference_dict[rece][invi] > preference_dict[rece][current_best]:
                            current_best = invi
                    paired[rece] = current_best
                
            # already paired receiver will compare the current match with the best in invitations
            elif getCurrMatch(rece, paired) != -1:
                # find the ones who send the invitation from the possible matches
                poss_invi = []
                for i in received[rece]:
                    if i in list(preference_dict[rece].keys()):
                        add_in = i
                        poss_invi.append(add_in)
            
                # find the best one among the possible choices in the invitations
                if poss_invi:
                    current_best = poss_invi[0]
                    for invi in poss_invi:
                        if preference_dict[rece][invi] > preference_dict[rece][current_best]:
                            current_best = invi
                    # if the receiver gets a better choice then current match, replace it
                    curr = getCurrMatch(rece, paired)                 
                    if preference_dict[rece][current_best] > preference_dict[rece][curr]:
                            if rece in list(paired.keys()):
                                paired[rece] = current_best
                            elif rece in list(paired.values()):
                                del paired[curr]
                                paired[rece] = current_best
            

In [7]:
def recursiveFind(preference_dict, subject_pref, paired):
    findPair(subject_pref, paired)
    thisscore = overallScore(preference_dict, paired)
    
    nextscore = []
    for i in range(7):
        findPair(subject_pref, paired)
        nextscore.append(overallScore(preference_dict, paired))

    for j in nextscore: 
        if (j > thisscore):
            recursiveFind(preference_dict, subject_pref, paired)

In [8]:
recursiveFind(preference_dict, subject_pref, paired)

In [9]:
paired

{10: 259,
 21: 266,
 22: 281,
 23: 3,
 29: 317,
 32: 171,
 38: 1,
 42: 149,
 45: 199,
 57: 67,
 64: 424,
 68: 315,
 75: 344,
 77: 336,
 98: 77,
 103: 148,
 104: 238,
 107: 239,
 109: 218,
 116: 339,
 135: 192,
 136: 53,
 147: 374,
 155: 273,
 156: 0,
 162: 343,
 175: 366,
 177: 176,
 181: 110,
 184: 439,
 186: 428,
 188: 325,
 190: 56,
 194: 324,
 210: 290,
 212: 455,
 214: 49,
 216: 33,
 219: 72,
 226: 349,
 232: 457,
 235: 224,
 243: 271,
 248: 478,
 250: 446,
 254: 50,
 261: 355,
 267: 466,
 277: 141,
 285: 51,
 293: 382,
 310: 248,
 321: 139,
 342: 28,
 356: 41,
 365: 306,
 367: 346,
 370: 119,
 375: 487,
 385: 245,
 389: 267,
 410: 369,
 412: 492,
 413: 62,
 414: 488,
 416: 207,
 419: 442,
 422: 377,
 430: 223,
 431: 43,
 433: 179,
 435: 265,
 438: 459,
 443: 161,
 491: 436,
 493: 379,
 494: 275,
 499: 126,
 7: 251,
 17: 426,
 58: 400,
 70: 101,
 78: 4,
 118: 331,
 129: 381,
 144: 30,
 159: 55,
 222: 362,
 225: 394,
 228: 432,
 231: 82,
 233: 353,
 249: 425,
 252: 111,
 255: 26,
 

In [10]:
# get all paired and unpaired pep into a dictionary
no_match = []
for pep in list(preference_dict.keys()):
    if pep not in list(paired.keys()) and pep not in list(paired.values()):
        no_match.append(pep)

pair1 = list(paired.keys())
pair2 = list(paired.values())
pair1.extend(no_match)
no_match_val = ['-1'] * len(no_match)
pair2.extend(no_match_val)
        
out = pd.DataFrame()
out['Subject'] = pair1
out['Match'] = pair2

out.to_excel('finalmatch.xlsx', encoding = 'utf-8', index = False)

        


In [11]:
out

,Subject,Match
0,10,259
1,21,266
2,22,281
3,23,3
4,29,317
...,...,...
298,479,-1
299,485,-1
300,495,-1
301,497,-1
